In [2]:
from bs4 import BeautifulSoup
from datetime import datetime
from  urllib.parse import quote, urlencode, urlparse
import json
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)

import requests

def make_soup(content):
    return BeautifulSoup(content, 'html.parser')

In [26]:
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 " \
             "Safari/537.36 "

keywords=["ryzen", "5900x"]

headers = {
    'user-agent': USER_AGENT,
    # "accept-language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7"
}

url = f"https://www.amd.com/fr/direct-buy/fr"

session = requests.Session()
session.headers.update(headers)

session.cookies.set("pmuser_country", "fr", domain="www.amd.com")
resp = session.get(url)
resp.raise_for_status()


In [27]:
bs = make_soup(resp.content)

items = bs.select(".view-shop-product-search .shop-content")

result = []
for item in items:
    title = item.select_one(".shop-title").get_text().strip()
    item_url_path = item.select_one(".shop-details a").attrs["href"]
    in_stock = item.select_one(".shop-links button") is not None
    price = float(item.select_one('.shop-price').get_text().replace('€', '').replace('$', '').replace(',', '.').strip())
    result.append((title, price, in_stock, item_url_path))


print(result)

[('AMD Ryzen™ 9 3950X Processeur de Bureau', 739.51, True, '/fr/direct-buy/5358858000/fr'), ('Processeur de bureau AMD Ryzen™ 9 3900X', 458.21, False, '/fr/direct-buy/5335621900/fr'), ('AMD Radeon™ RX 6900 XT Graphics', 986.47, False, '/fr/direct-buy/5458374200/fr'), ('AMD Radeon™ RX 6800 XT Graphics', 641.02, False, '/fr/direct-buy/5458374100/fr'), ('AMD Radeon™ RX 6800 Graphics', 571.93, False, '/fr/direct-buy/5458374000/fr'), ('AMD RYZEN™ 9 5950X Processor', 788.86, False, '/fr/direct-buy/5450881400/fr'), ('AMD RYZEN™ 9 5900X Processor', 542.1, False, '/fr/direct-buy/5450881500/fr'), ('AMD RYZEN™ 7 5800X Processor', 443.4, False, '/fr/direct-buy/5450881600/fr'), ('AMD RYZEN™ 5 5600X Processor', 295.36, False, '/fr/direct-buy/5450881700/fr')]


In [ ]:
resp_json = resp.json()
pp.pprint(resp_json["produits"][0])

In [72]:
metadata = bs.find("script", attrs={'type':'application/ld+json'})
pp.pprint(json.loads(metadata.string))

{   '@context': 'http://schema.org/',
    '@type': 'Product',
    'brand': {'@type': 'Brand', 'name': 'EVGA'},
    'description': 'La carte graphique EVGA GeForce RTX 3080 XC3 GAMING '
                   'embarque 10 Go de mémoire vidéo de nouvelle génération '
                   "GDDR6X. Ce modèle overclocké d'usine bénéficie de "
                   "fréquences de fonctionnement élevées et d'un système de "
                   'refroidissement amélioré gage de fiabilité et de '
                   'performances à long terme.',
    'mpn': '10G-P5-3883-KR',
    'name': 'EVGA GeForce RTX 3080 XC3 GAMING',
    'offers': {   '@type': 'Offer',
                  'availability': 'http://schema.org/OutOfStock',
                  'price': '863.95',
                  'priceCurrency': 'EUR'}}


In [90]:
import re
item = items[0]
print(item.find(class_="description").get_text())
item_id = item.attrs["id"]
script_data = ''.join(s.string for s in bs.find_all("script", attrs={"src": None}))
re.search("#{id}.*?stock-wrapper.*?stock-([0-9])".format(id=item_id), script_data)[1]


EVGA GeForce RTX 3090 FTW3 GAMING
24 Go GDDR6X - HDMI/Tri DisplayPort - PCI Express (NVIDIA GeForce RTX 3090)


Call of Duty : Black Ops Cold War offert avec NVIDIA
                    




'9'

In [77]:
target_url = "https://www.materiel.net/product-listing/stock-price/"
# target_url = "https://shop.hardware.fr/stock-price/"
headers = {
    'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    # 'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'x-requested-with': 'XMLHttpRequest'
}
# payload = "json=%7B%22currencyISOCode3%22%3A%22EUR%22%2C%22offers%22%3A%5B%7B%22offerId%22%3A%22AR202009090100%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009090101%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012070098%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009090098%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009090099%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202009100088%22%2C%22marketplace%22%3Afalse%7D%2C%7B%22offerId%22%3A%22AR202012070099%22%2C%22marketplace%22%3Afalse%7D%5D%2C%22shops%22%3A%5B%7B%22shopId%22%3A-1%7D%5D%7D&shopId=-1&displayGroups=Web&shopsAvailability=%7B%22AR202009090100%22%3A%220%22%2C%22AR202009090101%22%3A%220%22%2C%22AR202012070098%22%3A%220%22%2C%22AR202009090098%22%3A%220%22%2C%22AR202009090099%22%3A%220%22%2C%22AR202009100088%22%3A%220%22%2C%22AR202012070099%22%3A%220%22%7D"
# payload = "json=%7B%22currencyISOCode3%22%3A+%22EUR%22%2C+%22offers%22%3A+%5B%7B%22offerId%22%3A+%22AR202009090101%22%2C+%22marketplace%22%3A+false%7D%2C+%7B%22offerId%22%3A+%22AR202009090099%22%2C+%22marketplace%22%3A+false%7D%2C+%7B%22offerId%22%3A+%22AR202012070098%22%2C+%22marketplace%22%3A+false%7D%2C+%7B%22offerId%22%3A+%22AR202009100088%22%2C+%22marketplace%22%3A+false%7D%2C+%7B%22offerId%22%3A+%22AR202009090098%22%2C+%22marketplace%22%3A+false%7D%2C+%7B%22offerId%22%3A+%22AR202009090100%22%2C+%22marketplace%22%3A+false%7D%2C+%7B%22offerId%22%3A+%22AR202012070099%22%2C+%22marketplace%22%3A+false%7D%5D%2C+%22shops%22%3A+%5B%7B%22shopId%22%3A+-1%7D%5D%7D&shopId=-1&displayGroups=Web&shopsAvailability=%7B%22AR202009090101%22%3A+%220%22%2C+%22AR202009090099%22%3A+%220%22%2C+%22AR202012070098%22%3A+%220%22%2C+%22AR202009100088%22%3A+%220%22%2C+%22AR202009090098%22%3A+%220%22%2C+%22AR202009090100%22%3A+%220%22%2C+%22AR202012070099%22%3A+%220%22%7D"
payload = {
    "json": json.dumps({
        "currencyISOCode3":"EUR",
        "offers":[
            {"offerId":"AR202009090101","marketplace":False},
            {"offerId":"AR202009090099","marketplace":False},
            {"offerId":"AR202012070098","marketplace":False},
            {"offerId":"AR202009100088","marketplace":False},
            {"offerId":"AR202009090098","marketplace":False},
            {"offerId":"AR202009090100","marketplace":False},
            {"offerId":"AR202012070099","marketplace":False}],
        "shops":[{"shopId":-1}]}),
    # "shopId": -1,
    # "displayGroups": "Web",
    # "shopsAvailability": json.dumps({
    #     "AR202009090101":"0",
    #     "AR202009090099":"0",
    #     "AR202012070098":"0",
    #     "AR202009100088":"0",
    #     "AR202009090098":"0",
    #     "AR202009090100":"0",
    #     "AR202012070099":"0"})
}
# encoded = urlencode(payload)
# decoded = urlparse(encoded)

resp = requests.post(target_url, data=payload, headers=headers)
resp.raise_for_status()

In [78]:
pp.pprint(resp.json())

{   'nbShopsAvailability': [],
    'price': {   'AR202009090098': '<div class="o-product__prices "><span '
                                   'class="o-product__price">849€<sup>95</sup></span></div>',
                 'AR202009090099': '<div class="o-product__prices "><span '
                                   'class="o-product__price">899€<sup>95</sup></span></div>',
                 'AR202009090100': '<div class="o-product__prices "><span '
                                   'class="o-product__price">989€<sup>95</sup></span></div>',
                 'AR202009090101': '<div class="o-product__prices "><span '
                                   'class="o-product__price">999€<sup>95</sup></span></div>',
                 'AR202009100088': '<div class="o-product__prices "><span '
                                   'class="o-product__price">929€<sup>95</sup></span></div>',
                 'AR202012070098': '<div class="o-product__prices "><span '
                                   'class="

In [104]:
import re
len(json["price"])
def is_in_stock(art):
    match = re.search(r"o-availability__value--stock_([0-9])", art)
    assert match
    return int(match[1]) <= 2

any(map(is_in_stock, json["stock"].values()))

False

In [4]:
a = [1, 2, 3]

b = list(a)

print(b)
b[0] = 10
print(a)
print(b)

c = list(10)

[1, 2, 3]
[1, 2, 3]
[10, 2, 3]


TypeError: 'int' object is not iterable